In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split # деление на обучающую и проверочную выборку
from sklearn.preprocessing import OneHotEncoder # кодирует целочисленные значения у в массив
from sklearn.metrics import accuracy_score # подсчет точности предсказаний

# Подготовка данных

In [2]:
df = pd.read_csv('data/train.csv')

df_X = df.drop('label', axis=1) 
df_y = df['label']

train_x, test_x, train_y, test_y = train_test_split(df_X, df_y, test_size=0.3, random_state=20) 

# отложенные данные для теста
X_test = np.array(test_x) / 255 #  масштабируем чтобы значения были в даипозоне [0 - 1]
y_test = np.array([test_y]).T

# данные для обучения модели 
X = np.array(train_x) / 255 #  масштабируем чтобы значения были в даипозоне [0 - 1]
y = np.array([train_y]).T
y_onehotenc = OneHotEncoder().fit_transform(y).toarray()


'\nПосле всех преобразований получили размерности:\nX - [29400x784]\ny - [29400x1]\ny_onehotenc - [29400x10]\n\nx_test - [12600x784]\ny_test - [12600x1]\n'

# Функция активации и ее производная

In [3]:
# ф-я активации, при флаге True возвращает производную (для подсчета градиента)
def sigmoid(x, derivative=False):
    if derivative == True:
        return x * (1 - x)
    return 1 / (1 + np.exp(-x))

# Функция ошибки и ее производная

In [4]:
# ф-я ошибки, при флаге True возвращает производную (для подсчета градиента)
def loss(y, y_, derivative=False):
    if derivative == True:
        return y - y_
    return (y - y_)**2

# Математическое обоснование

Описание параметров:
 - Ф-я активации (activation) - $SIGMOID$ (logistic regression)
 - Ф-я ошибки (loss) - $MSE$ (mean squared error)
 - Оптимизатор (optimizer) - $SGDm$ (Stochastic gradient descent)
 - Кол-во эпох обучения (epoch) - 3000
 - Размер батча обучения (batch) - 32
 - Размер шага обучения (learning rate) - 1e-4

Ф-я активации СИГМОИД ($A$): $ \frac{1}{(1+e^{-\alpha})}$, где $\alpha$ считается как $ X_{input} * w + b$

Производная ф-ии активации: $dA = \alpha*(1-\alpha)$

Ф-я ошибки MSE ($E$): $\frac{\sum{(y-y^{'})^2}}{n}$, где $y$ - правильный ответ, а $y^{'}$ - предсказания сети

Производная ф-ии ошибки MSE: $dE = \frac{-2 (y-y^{'})}{n}$

Рекуретное отношение последнего слоя: $ \delta_{n} = dE_n * dA_n$

Рекуретное отношение скрытого слоя: $ \delta_{n} = \delta_{n+1} * w_{n+1} * dA$

Частная производная ф-ии ошибки по весам $w$:  $\frac{dE}{dw} = \delta * X_{input}$ --- градиент весов


Частная производная ф-ии ошибки по биасам $b$:  $\frac{dE}{db} = \delta$  --- градиент биасов

Уравнение обновления весов и биасов ($SGD$): $\omega = \omega - \alpha * \triangledown \omega$ => $\omega = \omega + \triangledown \omega$, т.к. антиградиент с "-" и $-\alpha$  дают "+". $\alpha$ заменяем learning rate

# Архитектура и обучение NN (2 слоя)

In [178]:
epoches = 50 # кол-во эпох обучения
batch = 32 # размер батча
lr = 1e-1 # learning rate
momentum = 0.3 # коэффициент инерционности 
dE_dw1_old = 0
dE_db1_old = 0
dE_dw2_old = 0
dE_db2_old = 0


w1 = np.random.random((784, 128)) / 1e4   # рандомно создаем матрицу весов для 1-го слоя и масштабируем
w2 = np.random.random((128, 10)) / 1e4   # рандомно создаем матрицу весов для 2-го слоя и масштабируем

b1 = np.zeros((1,128))  # создаем матрицу нулевых биасов для 1-го слоя
b2 = np.zeros((1,10))  # создаем матрицу нулевых биасов для 2-го слоя


for _ in range(epoches):
    for i in range(X.shape[0] // batch):
        batch_ids = np.random.choice(X.shape[0], batch, replace=False)
        
        y1 = sigmoid(X[batch_ids] @ w1 + b1) # предсказание 1-го слоя [32 x 128]
        y2 = sigmoid(y1 @ w2 + b2) # предсказание 2-го слоя [32 x 10]
        dE = loss(y_onehotenc[batch_ids], y2, True)  # берем производную по ф-ии ошибки


        # анти-градиент 2 слоя
        tmp2 = dE * sigmoid(y2, True) # рекурентное соотношение
        dE_dw2 = y1.T @ tmp2 / batch # градиент последнего слоя с нормированием по размеру батча
        dE_db2 = np.mean(tmp2, axis=0) # градиент биасов последнего слоя

        # анти-градиент 1 слоя
        tmp1 = (tmp2 @ w2.T) * sigmoid(y1, True) # рекурентное соотношение
        dE_dw1 = X[batch_ids].T @ tmp1 / batch # градиент скрытого слоя нормированный по размеру батча
        dE_db1 = np.mean(tmp1, axis=0) # градиент биасов скрытого слоя

        # изменяем веса и биасы
        w1 += lr * (dE_dw1 + momentum * dE_dw1) 
        b1 += lr * (dE_db1 + momentum * dE_db1)
        w2 += lr * (dE_dw2 + momentum * dE_dw2)
        b2 += lr * (dE_db2 + momentum * dE_db2)
        
        dE_dw1_old = dE_dw1 
        dE_db1_old = dE_db1
        dE_dw2_old = dE_dw2
        dE_db2_old = dE_db2

    
    if (_ % 5) == 0:  # показываем MSE каждые 100 эпох обучения
        error = y_onehotenc - sigmoid(sigmoid(X @ w1 + b1) @ w2 + b2)
        print(f"Epoch {_}. Error:" +  str(np.mean(np.abs(error))))
    
        # считаем точность на обученной выборке
        train_predict_matrix =  sigmoid(sigmoid(X @ w1 + b1) @ w2 + b2)
        train_predict_vector = train_predict_matrix.argmax(axis=1).T
        print("Train score: ", accuracy_score(y, train_predict_vector))
        
        # проверка точности на отложенной выборке
        test_predict_matrix = sigmoid(sigmoid(X_test @ w1 + b1) @ w2 + b2)
        test_predict_vector = test_predict_matrix.argmax(axis=1).T
        print("Test score: ",  accuracy_score(y_test, test_predict_vector))   # without bias 0.901984126984127
        
        # разделитель
        print("___________________________________________")

Epoch 0. Error:0.17994292153340974
Train score:  0.10306122448979592
Test score:  0.10484126984126985
___________________________________________
Epoch 5. Error:0.16951151242966628
Train score:  0.314421768707483
Test score:  0.3142857142857143
___________________________________________
Epoch 10. Error:0.08556900707733998
Train score:  0.845578231292517
Test score:  0.8425396825396826
___________________________________________
Epoch 15. Error:0.052586161909163646
Train score:  0.9007142857142857
Test score:  0.8973015873015873
___________________________________________
Epoch 20. Error:0.04278752083364778
Train score:  0.9121768707482993
Test score:  0.9092857142857143
___________________________________________
Epoch 25. Error:0.03704159577529341
Train score:  0.9206122448979592
Test score:  0.9157936507936508
___________________________________________
Epoch 30. Error:0.0336533898407888
Train score:  0.9266326530612244
Test score:  0.9215873015873016
_______________________________